In [2]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

In [9]:

from dotenv import load_dotenv
load_dotenv() 

True

In [1]:
pdf_path = Path("./Data Analysis With Python & Pandas.pdf")

NameError: name 'Path' is not defined

In [11]:

load = PyPDFLoader(file_path = pdf_path)
doc = load.load()


In [12]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

In [26]:
spited = text_splitter.split_documents(documents=doc)

In [27]:
import os
if not os.path.exists("qdrant_store"):
    store = Chroma.from_documents(
    documents=spited,
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
    persist_directory="qdrant_store"
    )
    print("New Chroma DB created.")
else:
    print("Directory already exists. Skipping creation.")

Directory already exists. Skipping creation.


In [28]:
retriever = store.as_retriever(
     search_type = "similarity",
        search_kwargs = {
            "k":10 
        }
)

In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [33]:
system_prompt = (
    "You are a helpful assistant that answers questions about Data Science {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)

In [34]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)    

In [ ]:
response = rag_chain.invoke({"input": "Can you explain Data Frame in Pandas briefly?"})
print(response["answer"])